In [1]:
import cv2
import numpy

## Step #1
Loads image and doubles it in size
image is displayed when run 

In [2]:
# displays original image and image doubled in size for processing
img = cv2.imread('melon_cat.jpg')
cv2.imshow('original', img)
# cv2.waitKey(0)
# receives height, width, channels
# print(img)
print(img.shape)
# goal is to have 4 octaves 5 blur levels
# if original image is first doubled in size you produce 4x as many keypoints
# pass back as width, height
double = (img.shape[1]*2, img.shape[0]*2)
print(double)
# does more or less the same thing, didn't see a visible difference
# img_doubled = cv2.resize(img, double, interpolation = cv2.INTER_LINEAR)
img_doubled = cv2.resize(img, double)
cv2.imshow('doubled in size', img_doubled)
cv2.waitKey(0)
cv2.destroyAllWindows()

(400, 468, 3)
(936, 800)


## Step #2
Creates scale images
generates an image then progressively blurs and rescale it
for each of the octaves I rescale by halving the image size each time

Each octave has 5 images the scale is increased since the blur is increased
documents says 4 octages 5 blur levels is the recommended way 

In [3]:
img = cv2.imread('melon_cat.jpg')

# each octaves image size is half of the original
# let's say that octave_num is 1-4

result = []

def octave(octave_num):
    # start off doubling image size for more key points
    width = img.shape[1]*2
    height = img.shape[0]*2
    if octave_num == 1:
        dim = (width, height)
    else:
        # image size is halved by each octave
        new_width = int(width*(1/(2*octave_num)))
        new_height = int(height*(1/(2*octave_num)))
        dim = (new_width, new_height)
    image = cv2.resize(img, dim)
    blurs = [(15,15), (19,19), (23,23), (25,25), (29,29)]
    count = 0
    for blur in blurs:
        result.append(cv2.GaussianBlur(image, blur, 0))

    for item in result:
        cv2.imshow('octave'+str(octave_num)+'blur'+str(count), item)
        count = count + 1
    print(count)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
octave(2)

5


## Step #3
DOG - Difference of Gaussian 
LoG - Laplacian of Gaussian
takes the scale images of the octave and subtracts them from each other in order to come up with the LoG approximation for key point generation 

In [4]:
# more features = more false positive results
# FIX BUG HERE CAUSING KERNEL TO CRASH
# use a python debugger
diff = []
def DoG():
    for i in range(len(result)-1):
        diff.append(result[i]-result[i+1])

    count = 0
    for item in diff:
        cv2.imshow('dog'+str(count), item)
        count = count + 1
#     return diff
# shows diff
    cv2.waitKey(0)
    cv2.destroyAllWindows()
DoG()

## Step #4 Keypoint detection
find the neighbors of each pixel and determine the minima and maxima these will be the keypoints
*note when comparing neighbors you need to look at the upper and lower layers also of each pixel 

DoG displays the difference of Gaussian so don't be deceived and think that it is the neighbors you are looking for

I am receiving a Nonetype error in my return for the neighbors which is unexpected

The goal of this section of code is to simply find the neighbors of the pixel and return it, but really I think i should be using 'diff' which is the approximated difference of the Laplacian of Gaussian

I am retrieving the positions that I wanted, but the positions are nothing like what I expected
Error: error stems from passing in a pixel instead of a position as I had expected thus it looks like the value of position (0,0) contained the pixel [10 10 10] thus the position was assumed to be (10,10)

In [43]:
# find neighbors for maxima and minima
# maxima and minima makrs keypoints
# keypoints not detected in uppermost and lowermost layers

# dog = DoG()
width = img.shape[1]
height = img.shape[0]
def find_neighbors(position, include):
    all_neighbors = []
    x = position[0]
    y = position[1]
    
    all_neighbors.append((x, y - 1))
    all_neighbors.append((x, y + 1))
    all_neighbors.append((x - 1, y))
    all_neighbors.append((x + 1, y))
    all_neighbors.append((x - 1, y - 1))
    all_neighbors.append((x + 1, y - 1))
    all_neighbors.append((x - 1, y + 1))
    all_neighbors.append((x + 1, y + 1))
    
    neighbors = []
    for i in all_neighbors:
        if (i[0] >= width) or (i[1] >= height):
            continue
        elif (i[0] >= 0) and (i[1] >= 0):
            neighbors.append(i)
    if(include):
        neighbors.append(position)
    return neighbors
print(diff[0][(0,0)]) # returns [10 10 10]
print(diff[0][0][0]) # returns [10 10 10]
print(find_neighbors((1,1), True))
print('trying to print position', find_neighbors((0,0), False))
print(find_neighbors(diff[0][(0,0)], True))
sol = find_neighbors(diff[0][(0,0)], True)
print(len(sol))
print(sol[len(sol) - 1])
# print(find_neighbors(dog[0][(0,0)], True))
# print(find_neighbors(dog[1][(0,0)], False))
# print(find_neighbors(dog[2][(0,0)], True))
# find_neighbors((0,0), False)

[10 10 10]
[10 10 10]
[(1, 0), (1, 2), (0, 1), (2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 1)]
trying to print position [(0, 1), (1, 0), (1, 1)]
[(10, 9), (10, 11), (9, 10), (11, 10), (9, 9), (11, 9), (9, 11), (11, 11), array([10, 10, 10], dtype=uint8)]
9
[10 10 10]


In [ ]:
# find keypoints from DOG using neighbors
# MUST pass in type of position in order for this to work
def get_keypoints(location):
    dog = DoG()
    check = []
    for index, layer in enumerate(dog):
        if index == 0 or index == len(dog):
            # skip uppermost & lowermost
            continue
        else:
            lower = find_neighbors(dog[index-1][location], True)
            point = find_neighbors(location, False)
            higher = find_neighbors(dog[index+1][location], True)
            check.extend(lower)
            check.extend(point)
            check.extend(higher)
            return check
get_keypoints((0,0)) # returns 8 

### Wrapper for keypoint finding
#### Code prior to this was minified testers in order to find the keypoints
##### Things to keep in mind:
- needs to run on ALL octaves, previous DoG only tested on one octave at time 
- needs to run on each pixel 
- width and height changes based on octave size
- need to retrieve all octaves in here 
- SKIP lowermost and topmost layer
-  cv2.circle(image, center_coordinates, radius, color, thickness)
<br></br>
#### Pixel dilemma 
Can either save pixels onto a separate list or color it I suppose

### Putting the code all together


In [5]:
# ROUGH DRAFT FOR PUTTING THINGS TOGETHER


# find_neighbor needsto run on ALL pixels of DOG

# diff is the DoG output, accepts the array of DoG 
# width & height changes based on each octave

def call_find_neighbors(diff):
    # retrieve one instance of the DoG outputs, dimension of the diffs will ALL be the same since it is from the same octave
    octave = diff[0]
    width = octave.shape[1] 
    height = octave.shape[0]
    print(octave_width, octave_height)
    for row in range(height):
        for col in range(width):
            find_neighbor((row,col), )
        
call_find_neighbors(diff)

234 200
234 200
234 200
234 200


In [1]:
# workshop on how I'm storing things
a = []
a.append((1,(3,4)))
print(a)
for item in a:
  print(item[0])
  print(item[1][0])
  print(item[1][1])

[(1, (3, 4))]
1
3
4


In [13]:
import cv2
import numpy


# returns list of neighbors that needs to be checked
def find_neighbors(index, dim):
    all_neighbors = []
    x, y = index
    width, height = dim

    all_neighbors.append((x, y - 1))
    all_neighbors.append((x, y + 1))
    all_neighbors.append((x - 1, y))
    all_neighbors.append((x + 1, y))
    all_neighbors.append((x - 1, y - 1))
    all_neighbors.append((x + 1, y - 1))
    all_neighbors.append((x - 1, y + 1))
    all_neighbors.append((x + 1, y + 1))

    neighbors = [index]
    #     save the location that the pixel is on for comparison
    for i in all_neighbors:
        if (i[0] >= width) or (i[1] >= height):
            continue
        elif (i[0] >= 0) and (i[1] >= 0):
            neighbors.append(i)
    return neighbors


def sift():
    img = cv2.imread('C:/Users/77jen/notebook/melon_cat.jpg')

    #  receives height, width, channels from .shape()
    width = img.shape[1]
    height = img.shape[0]

    # pass back as width, height
    # dim = (width, height)

    # double original image size
    double_dim = (width * 2, height * 2)
    img_doubled = cv2.resize(img, double_dim)

    # generate scale images
    scale_images = []
    for octave in range(1, 6):
        result = []
        if octave == 1:
            # uses the doubled image
            image = img_doubled
        else:
            # image size is halved by each octave
            new_width = int(width * (1 / (2 * octave)))
            new_height = int(height * (1 / (2 * octave)))
            new_dim = (new_width, new_height)
            image = cv2.resize(img, new_dim)
        blurs = [(15, 15), (19, 19), (23, 23), (25, 25), (29, 29)]
        for blur in blurs:
            result.append(cv2.GaussianBlur(image, blur, 0))
        scale_images.append(result)
    #     print(len(scale_images[0]), len(scale_images))
    #     print(scale_images[0])
    #     print(len(scale_images))

    # find DoG
    difference = []
    for octave in range(len(scale_images)):
        octave_val = scale_images[octave]
        diff = []
        for blur in range(len(octave_val) - 1):
            diff.append(octave_val[blur] - octave_val[blur + 1])
        difference.append(diff)
    #     print(difference[0])
    #     cv2.imshow('keypoints in octave', difference[0])
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()
    #     print(type(difference))

    # find keypoint
    keypoints = []
    keypoint_locations = []

    # level is the differently sized images, they contain 5 blurred images
    for size in range(len(difference)):
        for blur in range(len(difference[size])):
            if blur == 0 or blur == len(difference[size]):
                # skip uppermost & lowermost
                continue
            # instance of image
            instance = difference[size][blur]
            print(instance)
            # im a bit confused about the dimensions of this
            #         width = instance.shape[1]
            #         height = instance.shape[0]
            width = len(instance[0])
            height = len(instance[1])
            instance_dim = (width, height)
            # go through each pixel in the DoG
            for row in range(height):
                for col in range(width):
                    # compare with neighbors if max or min add it to keypoints & also save the location
                    neighbors_to_check = []
                    current_location = (row, col)
                    upper = difference[size][blur + 1]
                    currentPixel = instance[row][col]
                    #                 print(currentPixel.shape)
                    #                 print(numpy.array(upper).shape)
                    #                 print(currentPixel)
                    lower = difference[size][blur - 1]
                    # retrieves all valid neighbors to check, upper & lower neighbor check positions will be exactly the same
                    neighbors = find_neighbors(current_location, instance_dim)
                    for position in neighbors:
                        neighbors_to_check.append(upper[position[0]][position[1]])
                        neighbors_to_check.append(lower[position[0]][position[1]])
                    #                 print(neighbors_to_check[0])
                    #                 print('checking print')
                    #                 print(currentPixel)
                    #                 print()
                    #                 print(max(neighbors_to_check))
                    if not (currentPixel == max(neighbors_to_check) or currentPixel == min(neighbors_to_check)):
                        keypoints.append(currentPixel)
                        keypoint_locations.append(current_location)
        print()
        print(keypoints)


#                 pixel locations/neighbors
#                 locations = find_neighbors((row,col), instance_dim)
#                 for location in locations:
#                     check = upper[location[0]][location[1]]
#                     if(check > currentPixel || check < currentPixel):
#                         keypoints.append(check)
# #                         adds the level and the location of the pixel in the array
#                         keypoint_locations.append((upper, location))

#     # mark all keypoints
#     violet = (180, 0, 180)
#     l = 1
# #     print('number of keypoints:', len(keypoints.values()))
# #     for l, point in keypoints.items()
# #     w = difference[l].shape[1]
# #     h = difference[l].shape[0]
#     image = cv2.resize(img, (width,height))
#     circle = cv2.circle(image, point, 1, violet, -1)
#     cv2.imshow('keypoints in octave', l)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

sift()


[array([[[6, 6, 6],
        [6, 6, 6],
        [6, 6, 6],
        ...,
        [7, 7, 7],
        [8, 8, 8],
        [7, 7, 7]],

       [[5, 5, 5],
        [6, 6, 6],
        [6, 6, 6],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [8, 8, 8]],

       [[6, 6, 6],
        [6, 6, 6],
        [5, 5, 5],
        ...,
        [6, 6, 6],
        [6, 6, 6],
        [7, 7, 7]],

       ...,

       [[1, 1, 1],
        [2, 2, 2],
        [2, 2, 2],
        ...,
        [5, 5, 5],
        [5, 5, 5],
        [5, 5, 5]],

       [[2, 2, 2],
        [1, 1, 1],
        [2, 2, 2],
        ...,
        [5, 5, 5],
        [6, 6, 6],
        [6, 6, 6]],

       [[2, 2, 2],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [6, 6, 6],
        [6, 6, 6],
        [6, 6, 6]]], dtype=uint8), array([[[6, 6, 6],
        [7, 7, 7],
        [6, 6, 6],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [8, 8, 8]],

       [[7, 7, 7],
        [7, 7, 7],
        [6, 6, 6],
        ...,

ValueError: operands could not be broadcast together with shapes (936,3) (78,3) 

### Draw and mark key point

In [8]:
def markPixel():
    # rgb color spec
    violet = (180, 0, 180)
    
    img = cv2.imread('melon_cat.jpg')
    center = (int(img.shape[1]/2), int(img.shape[0]/2))
    
    circle = cv2.circle(img, center, 1, violet, -1)
    cv2.imshow('red panda', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
markPixel()

In [8]:
# find max/find min
def find_keypoint(options):
    for layer in dog:
        for pixel in layer:
            print(pixel)
            get_keypoints(pixel)
    print(options)
    
def find_min_max(options):
    max = None
    min = None
    for item in options:
        if max is None and min is None:
            max = item
            min = item
        if item > max:
            max = item
        if item < min:
            min = item
        return (min, max)